In [4]:
import numpy as np # Библиотека работы с массивами
import os
from abc import abstractmethod # Позволяет указывать на абстрактные методы

In [5]:
def load_data(folder): # Загрузка numpy массивов
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))    
    x_test = np.load(os.path.join(folder, 'x_test.npy'))    
    y_test = np.load(os.path.join(folder, 'y_test.npy'))    
    return x_train, y_train, x_test, y_test

In [6]:
def assert_preds_correct(your_preds, sklearn_preds) -> bool: # Оценка корректности предсказанных классов
    return np.abs(your_preds - sklearn_preds).sum() == 0

In [7]:
def assert_probs_correct(your_probs, sklearn_probs) -> bool: # Сравнение схожести вероятности принадлежности к классам к аналогичным из sklearn
    return np.abs(your_probs - sklearn_probs).mean() < 1e-3

In [8]:
class NaiveBayes:
    def __init__(self, n_classes): # Храним n классов
        self.n_classes = n_classes
        self.params = dict() # Параметры в словаре
        
    # --- PREDICTION ---
        
    def predict(self, x, return_probs=False): # Функция предсказания
        """
        x - np.array размерности [N, dim], 
        где N - количество экземпляров данных, 
        dim -размерность одного экземпляра (количество признаков).
        
        Возвращает np.array размерности [N], содержащий номера классов для
        соответствующих экземпляров.
        """
        preds = []
        for sample in x:
            preds.append(
                self.predict_single(sample, return_probs=return_probs) # Каждый экзепляр проходит через предикцию и получаем ответ
            )
        
        if return_probs:
            return np.array(preds, dtype='float32')
        
        return np.array(preds, dtype='int32')
    
    def predict_single(self, x, return_probs=False) -> int: 
        """
        Делает предсказание для одного экземпляра данных.
        
        x - np.array размерности dim.
        
        Возвращает номер класса, которому принадлежит x.
        """
        assert len(x.shape) == 1, f'Expected a vector, but received a tensor of shape={x.shape}'
        marginal_prob = self.compute_marginal_probability(x)  # P(x) - безусловная вероятность появления x
        
        probs = []
        for c in range(self.n_classes):                 # c - номер класса
            prior = self.compute_prior(c)               # P(c) - априорная вероятность (вероятность появления класса)
            likelihood = self.compute_likelihood(x, c)  # P(x|c) - вероятность появления x в предположении, что он принаждлежит c
            
            # Используем теорему Байесса для просчёта условной вероятности P(c|x)
            # P(c|x) = P(c) * P(x|c) / P(x)
            prob = prior * likelihood / marginal_prob
            probs.append(prob)
            
        if return_probs:
            return probs
        
        return np.argmax(probs)
    
    # Вычисляет P(x) - безусловная вероятность появления x.
    @abstractmethod
    def compute_marginal_probability(self, x) -> float:
        pass
    
    # Вычисляет P(c) - безусловная вероятность появления класса c (априорная вероятность).
    @abstractmethod
    def compute_prior(self, c) -> float:
        pass
    
    # Вычисляет P(x|c) - условная вероятность появления x в предположении, что он принаждлежит c.
    @abstractmethod
    def compute_likelihood(self, x, c) -> float:
        pass
    
    # --- FITTING ---
    
    def fit(self, x, y):
        self._estimate_prior(y)
        self._estimate_params(x, y)
    
    @abstractmethod
    def _estimate_prior(self, y):
        pass
    
    @abstractmethod
    def _estimate_params(self, x, y):
        pass

## 1. Наивный классификатор Байеса: гауссово распределение

Напишите недостающий код, создайте и обучите модель. 

Пункты оценки:
1. совпадение предсказанных классов с оными у модели sklearn. Для проверки совпадения используйте функцию `assert_preds_correct`.
2. совпадение значений предсказанных вероятностей принадлежности классами с оными у модели sklearn. Значения вероятностей считаются равными, если функция `assert_probs_correct` возвращает True.

In [9]:
x_train, y_train, x_test, y_test = load_data('gauss')

In [10]:
from scipy.special import logsumexp # Данная функция 

In [11]:
class NaiveGauss(NaiveBayes):
    def __init__(self, n_classes, logs_variant=False):
        """
        Получим Наивный Байес, который имеет распределение Гаусса
        Тогда можно найти параметры, такие как математическое ожидание, дисперсия и так далее
        """
        super().__init__(n_classes)
        self.logs_variant = logs_variant
        
    def fit(self, x, y):
        n_features = x. shape[1]
        # Инициализация переменных
        self.params['class_count'] = np.zeros(self.n_classes, dtype=np.float32) # Частота
        self.params['prior'] = np.zeros(self.n_classes, dtype=np.float32)
        self.params['theta'] = np.zeros((self.n_classes, n_features), dtype=np.float32) # Математическое ожидание
        self.params['var'] = np.zeros((self.n_classes, n_features), dtype=np.float32) # Дисперсия
        self.params['std'] = np.zeros((self.n_classes, n_features), dtype=np.float32) # Корень из дисперсии
        
        # Значения априорных вероятностей сохраните в 'params' с ключом 'prior'
        # Напишите свой код здесь
        for y_i in np.unique(y): # Для каждого класса считаем статистики, которые в дальнейшем используем
            # y - количество вхождений определенного класса
            # Получаем индексы всех уникальных классов, которые используем в выборке
            # Берем выборку с известным классом
            x_i = x[y == y_i] 
            self.params['theta'][y_i] = np.mean(x_i, axis=0)
            self.params['var'][y_i] = np.var(x_i, axis=0)
            self.params['std'][y_i] = np.std(x_i, axis=0)
            self.params['class_count'][y_i] = np.sum(y == y_i)
        self.params['prior'][:] = self.params['class_count'] / np.sum(self.params['class_count'])
    
    
    # Вычисляет P(x) - безусловная вероятность появления x
    def compute_marginal_probability(self, x) -> float:
        # Для просчёта безусловной вероятности используйте 
        # методы compute_prior и compute_likelihood.
        # Напишите свой код здесь
        # Z = sum(p(c_i) * p(x| c_i))
        return np.sum(
            [               # p(c_i) * p(x | c_i)
                self.compute_prior(y_i) * self.compute_likelihood(x, y_i)
                for y_i in range(self.n_classes)
            ]
        )
    
    # Вычисляет P(c) - безусловная вероятность появления класса c (априорная вероятность)
    def compute_prior(self, c) -> float:
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        # Напишите свой код здесь
        return self.params['prior'][c]
    
    # Вычисляет P(x|c) - условная вероятность появления x в предположении, что он принаждлежит c.
    def compute_likelihood(self, x, c) -> float:
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        # Напишите свой код здесь
        var = self.params['var'][c][None, :] # Используем ранее рассчитанные значения
        std = self.params['std'][c][None, :]
        theta = self.params['theta'][c][None, :]
        
        # В этом примере N = 1 для x, именно поэтому мы можем использовать функцию squeeze
        if self.logs_variant:
            return np.squeeze(self.likelihood_logs(x, var, std, theta)) # Функция squeeze() удаляет оси с одним элементом (длинной 1), но не сами элементы массива
        return np.squeeze(self.likelihood_scratch(x, var, std, theta))
    
    def likelihood_scratch(self, x, var, std, theta): # Проверяет входит ли экземпляр данных в распределение (формула плотности вероятности)
        # x - вектор признаков
        # Посчитаем плотность вероятности в "лоб" по сухим формулам
        res = 1 / (np.sqrt(2* np.pi) * std)
        res *= np.exp((-0.5) * (((x - theta) ** 2) / var))
        # (N, n_features) ->(N,)
        return np.prod(res, axis = 1)
    
    def likelihood_logs(self, x, var, std, theta): # Проверяет входит ли экземпляр данных в распределение (формула плотности вероятности),
                                                   # с помощью логарифмирования функции плотности вероятности Гауссовского распределения 
        # Посчитаем плотность вероятности
        res = -0.5 * np.sum(np.log(2.0 * np.pi * var))
        res *= 0.5 * np.sum(((x - theta) ** 2) / var)
        joint_log_likelihood = np.array(res).T # (N, n_features) ->(n_features, N)
        res = logsumexp(res, axis = 0) # (n_features, N) -> (N,)
        return np.exp(res)
    
    

In [12]:
# Создайте и обучите модель
model = NaiveGauss(len(np.unique(y_train)), logs_variant=False)
model.fit(x_train, y_train)

In [13]:
# Оцените качество модели
preds = model.predict(x_test)
np.mean(preds == y_test)

1.0

Баесовский классификатор написан верно, так как точность равна единице. Ошибок нет

In [14]:
# Сравните вашу модель с аналогом sklearn (GaussianNB)
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)

print(f'preds corrent? {assert_preds_correct(model.predict(x_test), gnb.predict(x_test))}')
print(f'probs corrent? {assert_probs_correct(model.predict(x_test, True), gnb.predict_proba(x_test))}')

preds corrent? True
probs corrent? True


По данному сравнению можно сделать вывод, что модель работает аналогично аналогу из sklearn

Теперь сравним работу модели, реализованной с помощью логарифмированния функции плотности вероятности

In [18]:
# Создайте и обучите модель
model = NaiveGauss(len(np.unique(y_train)), logs_variant=True)
model.fit(x_train, y_train)

In [19]:
# Оцените качество модели
preds = model.predict(x_test)
np.mean(preds == y_test)

1.0

Баесовский классификатор написан верно, так как точность равна единице. Ошибок нет

In [22]:
# Сравните вашу модель с аналогом sklearn (GaussianNB)
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)

print(f'preds corrent? {assert_preds_correct(model.predict(x_test), gnb.predict(x_test))}')
print(f'probs corrent? {assert_probs_correct(model.predict(x_test, True), gnb.predict_proba(x_test))}')

preds corrent? True
probs corrent? True


По данному сравнению можно сделать вывод, что модель работает аналогично аналогу из sklearn

## 2. Доп. задания (любое на выбор, опционально)

### 2.1  Упрощение наивного классификатора Байеса для гауссова распределения

Уберите из класса NaiveBayes 'лишние' вычисления и удалите код, что соответствует этим вычислениям. Под 'лишним' подразумеваются вещи, что не влияют на итоговое решение о принадлежности классу (значения вероятностей при этом могу стать некорректными, но в данном задании это допустимо).

Напишите в клетке ниже код упрощенного 'классификатора Гаусса' и убедитесь, что его ответы (не значения вероятностей) совпадают с ответами классификатора из задания 1.

Указание: работайте в предположении, что классы равновероятны.

In [ ]:
# Напишите обновленный код модели здесь

In [ ]:
# Создайте и обучите модель

In [ ]:
# Оцените качество модели

In [ ]:
# Сравните вашу модель с моделью из задания 1

### 2.1  Наивный классификатор Байеса: мультиномиальное распределения

Напишите недостающий код, создайте и обучите модель.

Подсказка: в определении функции правдоподобия много факториалов. Для избежания численного переполнения посчитайте сначала логарифм функции правдоподобия, после примените экспоненту для получения значения вероятности.

Пункты оценки аналогичны оным из задания 1.

Сложность: математический гений.

In [ ]:
x_train, y_train, x_test, y_test = load_data('multinomial')

In [ ]:
"""
При желании данный класс можно переписать с нуля. Изменения должны сопровождаться комментариями.
"""
class NaiveMultinomial(NaiveBayes):
    def compute_marginal_probability(self, x) -> float:
        # Для просчёта безусловной вероятности используйте 
        # методы compute_prior и compute_likelihood.
        # Напишите свой код здесь
        pass
    
    def compute_prior(self, c) -> float:
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        # Напишите свой код здесь
        pass
    
    def compute_likelihood(self, x, c) -> float:
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        # Напишите свой код здесь
        pass
    
    # --- FITTING ---
    
    def _estimate_prior(self, y):
        # Значения априорных вероятностей сохраните в `params` с ключом 'prior'
        # Напишите свой код здесь
        pass
    
    def _estimate_params(self, x, y):
        # Напишите свой код здесь
        pass

In [ ]:
# Создайте и обучите модель

In [ ]:
# Оцените качество модели

In [ ]:
# Сравните вашу модель с аналогом sklearn (MultinomialNB)